# Old but working

BALANCE:
- Always solvable in the time given, but not extravagant.
- possible to beat the zombies, but requires cleverness
- 


VARIANTS:
- can/can't place on zombie spawn point
- wrap around the edges of the map
- inverted controls
- zombie spawn intervals
- zombie duration
- number of balls
- board size
- location of target zone
- colors

# New

In [ ]:
import gym
from gym import spaces
from gym.utils import seeding
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from IPython.display import display, clear_output
from dataclasses import dataclass



@dataclass
class Agent:
    """
    Represents an agent in the game.
    """
    row: int
    col: int
    direction: str
    holding_block: str

@dataclass
class BallColor:
    """Represents a type of ball in the game."""
    ball_color: str
    background_color: str  # same color, but lighter
    color_char: str # background will be lowercase, ball will be uppercase
    min_num: int
    max_num: int  # sample from this range, inclusive
 

class GridGame(gym.Env):
    """
    A grid-based game environment compatible with OpenAI Gym.
    The game involves an agent that needs to move balls to a target zone while avoiding zombies.
    """
    metadata = {'render.modes': ['human', 'rgb_array']}
    
    def __init__(self,
                 grid_width:int=10,
                 grid_height:int=10,
                 ball_colors:list[BallColor]=[BallColor('red', 'mistyrose', 'r', 5, 5), BallColor('purple', 'thistle', 'p', 5, 5)],
                 zombie_prob:int=0.02,
                 num_zombie_steps:int=10):
        """
        Initialize the game environment.
        
        Parameters:
            grid_width (int): Width of the grid.
            grid_height (int): Height of the grid.
            num_red_blocks (int): Number of red blocks to be placed on the grid.
            num_purple_blocks (int): Number of purple blocks to be placed on the grid.
            zombie_prob (float): Probability of a zombie spawning at designated spawn points.
            num_zombie_steps (int): Number of steps a zombie can move before disappearing.
        """
        super(GridGame, self).__init__()
        self.grid_width = grid_width
        self.grid_height = grid_height
        self.ball_colors = ball_colors
        self.ball_char_to_dict = {ball_color.color_char: ball_color for ball_color in self.ball_colors}
        self.zombie_prob = zombie_prob
        self.num_zombie_steps = num_zombie_steps
        self.holding_block = False
        
        # Throw an error if the board is too small to hold the target zone and spawn points
        if self.grid_width < 4 or self.grid_height < 4:
            raise ValueError("Board is too small to hold the target zone and spawn points; expected at least 4x4, got "
                             f"{self.grid_width}x{self.grid_height}")  # TODO: what order?
        
        # Throw an error if any of the numbers are invalid
        if any(ball_color.min_num < 0 for ball_color in self.ball_colors):
            raise ValueError("Cannot have a negative number of blocks")
        if any(ball_color.max_num < ball_color.min_num for ball_color in self.ball_colors):
            raise ValueError("Cannot have a max number of blocks less than the min number of blocks")
        if any(ball_color.max_num > 9 for ball_color in self.ball_colors):
            raise ValueError("Cannot have more than 9 blocks of a single color (due to target zone size)")
        if self.zombie_prob < 0 or self.zombie_prob > 1:
            raise ValueError(f"Zombie probability must be between 0 and 1; got {self.zombie_prob}")
        if self.num_zombie_steps < 1:
            raise ValueError(f"Zombie steps must be at least 1; got {self.num_zombie_steps}")
        
        # Throw an error if any of the colors match each other
        if len(set(ball_color.color_char for ball_color in self.ball_colors)) != len(self.ball_colors):
            raise ValueError("Cannot have duplicate ball colors")
        
        # Throw an error if any of the colors match the zombie color
        self.zombie_color = 'green'
        self.zombie_background = 'lightgreen'
        self.zombie_char = 'z'
        ball_and_background_colors = [ball_color.color_char for ball_color in self.ball_colors] + [ball_color.background_color for ball_color in self.ball_colors]
        if any(color in ball_and_background_colors for color in [self.zombie_color, self.zombie_background]):
            raise ValueError(f"Cannot have a ball color that matches the zombie color ({self.zombie_color})")

        # Throw an error if there are too many blocks to fit in the target zone
        if max(ball_color.max_num for ball_color in self.ball_colors) > 9:
            raise ValueError("Too many blocks to fit in the target zone")
        
        
        # Initialize the game board and other game state variables
        
        # Board-ground is a 2D list of strings representing the ground layer of the board
        # "_" represents a blank square, "z" represents a zombie spawn point, and the target zone is represented by the target color
        self.empty_char = "_"
        self.board_ground = None
        # Board-objects is a 2D list of strings representing the objects layer of the board
        # "_" represents a blank square, the agent is represented by "^", ">", "<", or "v" depending on its direction, and blocks are represented by their color
        self.board_objects = None
        # List of zombies on the field
        self.zombies = []

        self.agent = None  # Agent object (will be initialized on reset)
        self.num_moves = 0  # Number of moves taken by the agent
        self.target_color = None  # Color of the target zone (will be initialized on reset)

        # Define the action space (4 moves: up, left, down, right)
        self.action_space = spaces.Discrete(4)
        
        # Define the observation space
        self.observation_space = spaces.Box(
            low=0, high=1, 
            shape=(grid_height, grid_width, 2),  # Two layers: ground and objects
            dtype=np.float32
        )

        # Gym-specific attributes
        self.seed()
        self.viewer = None
        
        # Call reset to initialize board and agent
        self.reset()

    def seed(self, seed=None):
        """
        Set the seed for this environment's random number generator.
        
        Parameters:
        seed (int or None): The seed to use. If None, a random seed will be used.
        
        Returns:
        list: A list containing the seed used.
        """
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def reset(self):
        """
        Reset the game environment to its initial state.
        
        Returns:
        np.array: The initial observation of the game state.
        """
        # Reset the game state
        self.initialize_board()
        self.place_agent()
        self.place_balls()
        self.zombies = []
        self.num_moves = 0
        self.holding_block = False
        
        # Return the initial observation
        return self.get_observation()
    
    def step(self, action):
        """
        Take an action in the environment.
        
        Parameters:
        action (int): An action to take, represented as an integer.
        
        Returns:
        tuple: A tuple containing:
            - np.array: The observation after taking the action.
            - float: The reward obtained after taking the action.
            - bool: Whether the game is over after taking the action.
            - dict: Extra information about the step.
        """
        # Map the discrete action to a game action and perform it
        action_map = {0: 'w', 1: 'a', 2: 'd', 3: 's'}
        game_action = action_map[action]
        self.move_agent(game_action)
        
        # Check if the game is over
        done = self.is_game_over()
        
        # Calculate the reward
        if any(zombie['position'] == (self.agent_row, self.agent_col) for zombie in self.zombies):
            reward = -10
        else:
            reward = 100 - self.num_moves if self.num_moves < 100 else 0
        
        # Return step information
        return self.get_observation(), reward, done, {}
    
    def get_observation(self):
        """
        Get the current observation of the game state.
        
        Returns:
        np.array: The current observation of the game state.
        """
        # Convert the board state to a binary representation for the observation
        ground_layer = np.array(self.board_ground) != self.empty_char
        objects_layer = np.array(self.board_objects) != self.empty_char
        return np.stack([ground_layer, objects_layer], axis=-1).astype(np.float32)
    
    def render(self, mode='human'):
        """
        Render the current game state.
        
        Parameters:
        mode (str): The mode to render with ('human' or 'rgb_array').
        """
        if mode == 'rgb_array':
            return self.get_image()
        elif mode == 'human':
            self.print_board_image()
    
    def close(self):
        """
        Perform any necessary cleanup.
        """
        if self.viewer:
            self.viewer.close()
            self.viewer = None

    def initialize_board(self):
        """
        Initialize the game board to its starting state.
        """
        # Initialize both boards full of clear ground
        self.board_ground = [[self.empty_char] * self.grid_width for _ in range(self.grid_height)]
        self.board_objects = [[self.empty_char] * self.grid_width for _ in range(self.grid_height)]
        
        # Set zombie spawn points on the ground board
        self.board_ground[0][0] = self.zombie_char.lower()
        self.board_ground[self.grid_height-1][self.grid_width-1] = self.zombie_char.lower()
        
        # Randomly decide where the target zone is and its color
        if np.random.choice([True, False]):  # Top-left
            row_start, col_start = 0, self.grid_width - 3
            print('Placing on the top left')
        else:  # Bottom-right
            print('placing on the bottom right')
            row_start, col_start = self.grid_height -3, 0
        self.target_color = np.random.choice([ball_color.color_char.lower() for ball_color in self.ball_colors])
        
        # Fill in the target zone on the ground board
        for i in range(3):
            for j in range(3):
                self.board_ground[row_start+i][col_start+j] = self.target_color

    def place_agent(self):
        """
        Place the agent on the game board at a random empty location.
        """
        agent_directions = ['^', '>', '<', 'v']
        self.agent_direction = np.random.choice(agent_directions)
        self.agent_row, self.agent_col = self.get_random_empty_square()
        self.board_objects[self.agent_row][self.agent_col] = self.agent_direction

    def place_balls(self):
        """
        Place balls on the game board at random empty locations.
        The number of balls of each color is sampled from the range specified in the BallColor objects.
        """
        for ball_info in self.ball_colors:
            for _ in range(np.random.randint(ball_info.min_num, ball_info.max_num + 1)):
                row, col = self.get_random_empty_square()
                self.board_objects[row][col] = ball_info.color_char.upper()

    def get_random_empty_square(self, timeout=1000):
        """Try up to timeout times to find an empty square.
        
        Parameters:
        timeout (int): The number of attempts to find an empty square.
        
        Returns:
        tuple: (row, col), both ints
        """
        for _ in range(timeout):
            row = np.random.randint(0, self.grid_height)
            col = np.random.randint(0, self.grid_width)
            if self.board_objects[row][col] == self.empty_char and self.board_ground[row][col] == self.empty_char:
                return row, col
        raise ValueError("Could not find an empty square")
                           
    def spawn_zombies(self):
        spawn_points = [(0, 0), (self.grid_height-1, self.grid_width-1)]
        for point in spawn_points:
            if self.board_objects[point[0]][point[1]] == self.empty_char and np.random.random() < self.zombie_prob:
                self.zombies.append({'position': point, 'remaining_steps': self.num_zombie_steps})
                self.board_objects[point[0]][point[1]] = self.zombie_char.upper()
                
    def move_zombies(self):
        for zombie in self.zombies:
            r, c = zombie['position']
            dr, dc = self.agent_row - r, self.agent_col - c
            
            # Determine the direction to move the zombie
            if abs(dr) >= abs(dc):
                r_move = min(dr, 2) if dr > 0 else max(-2, dr)
                c_move = 0
            else:
                c_move = min(dc, 2) if dc > 0 else max(dc, -2)
                r_move = 0
            
            # Adjust for boundary or block collisions
            print('=============')
            for r_check in range(1, abs(r_move) + 1):  # check that the path is clear
                if r_move < 0:
                    r_check *= -1
                    is_negative = True
                else:
                    is_negative = False
                if not (0 <= r + r_check < self.grid_height) or self.board_objects[r + r_check][c] in [ball_color.color_char.upper() for ball_color in self.ball_colors] + [self.zombie_char.upper()]:
                    # Stop right before this object
                    r_move = r_check + 1 if is_negative else r_check - 1
                    break
                # If the agent is there, move on top of the agnet
                if (r + r_check, c) == (self.agent_row, self.agent_col):
                    r_move = r_check
                    break
            for c_check in range(1, abs(c_move) + 1):  # check that the path is clear
                if c_move < 0:
                    c_check *= -1
                    is_negative = True
                else:
                    is_negative = False
                if not (0 <= c + c_check < self.grid_width) or self.board_objects[r][c + c_check] in [ball_color.color_char.upper() for ball_color in self.ball_colors] + [self.zombie_char.upper()]:
                    # Stop right before this object
                    c_move = c_check + 1 if is_negative else c_check - 1
                    break
                if (r, c + c_check) == (self.agent_row, self.agent_col):
                    c_move = c_check
                    break

            # Remove the zombie from its current position
            self.board_objects[r][c] = self.empty_char
            
            # Move the zombie
            r += r_move
            c += c_move
            
            # Check if the zombie lands on the agent
            if (r, c) == (self.agent_row, self.agent_col):
                self.board_objects[r][c] = self.zombie_char.upper()
            
            # Place the zombie in its new position and update its position in the list
            self.board_objects[r][c] = self.zombie_char.upper()
            zombie['position'] = (r, c)
            
            # Decrease the remaining steps for the zombie
            zombie['remaining_steps'] -= 1
            if zombie['remaining_steps'] == 0:
                self.board_objects[r][c] = self.empty_char
                self.zombies.remove(zombie)

    def move_agent(self, action):
        prev_row, prev_col = self.agent_row, self.agent_col
        self.num_moves += 1
        
        if action == "w":
            if self.agent_direction == '^':
                self.agent_row -= 1
            elif self.agent_direction == 'v':
                self.agent_row += 1
            elif self.agent_direction == '>':
                self.agent_col += 1
            elif self.agent_direction == '<':
                self.agent_col -= 1
                
            # Check for collisions or going out of bounds
            if (self.agent_row < 0 or self.agent_row >= self.grid_height or
                self.agent_col < 0 or self.agent_col >= self.grid_width or
                self.board_objects[self.agent_row][self.agent_col] not in [self.empty_char]):
                self.agent_row, self.agent_col = prev_row, prev_col
                self.move_zombies()
                self.spawn_zombies()
                return  # Invalid move
                
        elif action == "a":
            self.agent_direction = {'^': '<', 'v': '>', '>': '^', '<': 'v'}[self.agent_direction]
                
        elif action == "d":
            self.agent_direction = {'^': '>', 'v': '<', '>': 'v', '<': '^'}[self.agent_direction]
        
        elif action == "s":
            # Determine the square in front of the agent
            front_row, front_col = prev_row, prev_col
            if self.agent_direction == '^':
                front_row -= 1
            elif self.agent_direction == 'v':
                front_row += 1
            elif self.agent_direction == '>':
                front_col += 1
            elif self.agent_direction == '<':
                front_col -= 1

            # If the agent is holding a block
            if self.holding_block:
                # Try to place the block if the square in front is empty and inside the board
                if (0 <= front_row < self.grid_height and 0 <= front_col < self.grid_width and
                    self.board_objects[front_row][front_col] == self.empty_char):
                    self.board_objects[front_row][front_col] = self.holding_block
                    self.holding_block = None
            else:
                # If the square in front has a block, pick it up
                if self.board_objects[front_row][front_col] in [color.color_char.upper() for color in self.ball_colors]:
                    self.holding_block = self.board_objects[front_row][front_col]
                    self.board_objects[front_row][front_col] = self.empty_char
        
        # Update the objects board with the new agent position and direction
        self.board_objects[prev_row][prev_col] = self.empty_char
        self.board_objects[self.agent_row][self.agent_col] = self.agent_direction
        self.move_zombies()
        self.spawn_zombies()

    def is_game_over(self):
        if self.num_moves >= 500:
            return True
        if any(zombie['position'] == (self.agent_row, self.agent_col) for zombie in self.zombies):
            return True
        # Check if all objects of target color are in target zone
        correctly_placed_blocks = 0
        target_zone_count = 0
        for i in range(self.grid_height):
            for j in range(self.grid_width):
                if self.board_objects[i][j].upper() == self.target_color.upper():  # look for target objects
                    # We aren't finished if the target objs are outside the target zone
                    if self.board_ground[i][j] != self.target_color:
                        return False
                    else:
                        correctly_placed_blocks += 1                 
        # Game ends if all objects of target color are in target zone or player reaches 100 moves
        print('GAME OVER?', target_zone_count == 9 or self.num_moves >= 100 or any(zombie['position'] == (self.agent_row, self.agent_col) for zombie in self.zombies))
        return correctly_placed_blocks == self.num_target_blocks

    def print_board(self):
        for i in range(self.grid_height):
            row = []
            for j in range(self.grid_width):
                if self.board_objects[i][j] != self.empty_char:
                    row.append(self.board_objects[i][j])
                else:
                    row.append(self.board_ground[i][j])
            print(' '.join(row))

    def play_game(self, print_fn='text'):
        print("Welcome to the Grid Game!")
        if print_fn == 'text':
            self.print_board()
        elif print_fn == 'image':
            self.print_board_image()
        print("Commands: w (move), a (turn left), d (turn right), s (pick/place block), quit")
        
        while True:
            action = input("Enter your move: ").lower().strip()
            if action in ['w', 'a', 'd', 's']:
                self.move_agent(action)
                if print_fn == 'text':
                    self.print_board()
                elif print_fn == 'image':
                    self.print_board_image()
                else:
                    raise NotImplementedError
                print(f"Moves taken: {self.num_moves}")
                if self.is_game_over():
                    if any(zombie['position'] == (self.agent_row, self.agent_col) for zombie in self.zombies):
                        score = - 10
                    else:
                        score = 100 - self.num_moves if self.num_moves < 100 else 0
                    print(f"Game over! Your score: {score}")
                    break
            elif action == 'quit':
                print("Thanks for playing!")
                break
    
    def print_board_image(self):
        plt.close()
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(10,10))
        ax.set_xlim(0, self.grid_width)
        ax.set_ylim(0, self.grid_height)
        ax.set_xticks(range(self.grid_width + 1))
        ax.set_yticks(range(self.grid_height + 1))
        ax.grid(which='both')

        # Loop through all board ground cells
        for i in range(self.grid_height):
            for j in range(self.grid_width):
                color_char = self.board_ground[i][j]
                if color_char == self.empty_char:
                    continue
                if color_char == self.zombie_char:
                    color = self.zombie_background
                else:
                    color = self.ball_char_to_dict[color_char].background_color
                ax.add_patch(patches.Rectangle((j, self.grid_height-i-1), 1, 1, facecolor=color))

        # Draw blocks and agent
        for i in range(self.grid_height):
            for j in range(self.grid_width):
                obj_char = self.board_objects[i][j]
                if obj_char == self.empty_char:
                    continue
                if obj_char in ['^', 'v', '<', '>']:
                    if obj_char == '^':
                        points = [(j+0.5, self.grid_height-i), (j, self.grid_height-i-0.5), (j+1, self.grid_height-i-0.5)]
                    elif obj_char == 'v':
                        points = [(j+0.5, self.grid_height-i-1), (j, self.grid_height-i-0.5), (j+1, self.grid_height-i-0.5)]
                    elif obj_char == '>':
                        points = [(j+1, self.grid_height-i-0.5), (j+0.5, self.grid_height-i), (j+0.5, self.grid_height-i-1)]
                    elif obj_char == '<':
                        points = [(j, self.grid_height-i-0.5), (j+0.5, self.grid_height-i), (j+0.5, self.grid_height-i-1)]
                    triangle = patches.Polygon(points, closed=True, facecolor='blue')
                    ax.add_patch(triangle)
                else:
                    color = self.zombie_color if obj_char == self.zombie_char.upper() else self.ball_char_to_dict[obj_char.lower()].ball_color
                    ax.add_patch(patches.Circle((j+0.5, self.grid_height-i-0.5), 0.4, facecolor=color))
        display(plt.gcf())


In [ ]:
game.print_board()

In [ ]:
game = GridGame()
game.play_game('image')

-